In [32]:
import sys 
sys.path.append("../")

import torch 
import torch.nn

from Conv1d_NN import Conv1d_NN

from models import * 
from train import * 
from dataset import * 
from pixelshuffle import * 

Input:  torch.Size([32, 12, 40])
Upsampled:  torch.Size([32, 3, 160])
Downsampled:  torch.Size([32, 12, 40])
Inverse module works.


# Pixel shuffle for 1D 

In [2]:
import torch 
from torch import nn 

x = torch.rand(1, 32, 40, 40)
upscale_factor = 2

y = nn.functional.pixel_shuffle(x, upscale_factor)

print(x.shape)
print(y.shape)

torch.Size([1, 32, 40, 40])
torch.Size([1, 8, 80, 80])


# Neural Network testing with 5 nearest Neighbors

### Noisy Data

In [3]:
# Regular Noise Dataset
dataset = MNIST1D_Dataset() 
data = dataset.make_dataset()

### Less Noisy Data

In [4]:
less_noise_dataset = MNIST1D_Dataset()
less_noise_dataset.data_args.corr_noise_scale = 0.1
less_noise_dataset.data_args.iid_noise_scale = 0.01

less_noise_data = less_noise_dataset.make_dataset()

### Conv1d Regular Convolution Kernel = 5

In [21]:
reg_model = nn.Sequential(
   nn.Conv1d(in_channels=1, out_channels=32, kernel_size=5),
   nn.ReLU(), 
   nn.Conv1d(in_channels=32, out_channels=32, kernel_size=5),
   nn.ReLU(),
   nn.Conv1d(in_channels=32, out_channels=32, kernel_size=5),
   nn.ReLU(), 
   nn.Flatten(), 
   nn.Linear(896, 10)
)

# Get Summary 
from torchsummary import summary
# summary(reg_model, (1, 40), device='cpu')

In [6]:
reg_result = train_model(data, reg_model, dataset.model_args)

step 1000, dt 3.35s, train_loss 4.085e-04, test_loss 1.568e-01, train_acc 100.0, test_acc 96.9
step 2000, dt 2.89s, train_loss 5.531e-05, test_loss 1.548e-01, train_acc 100.0, test_acc 97.0
step 3000, dt 3.02s, train_loss 2.135e-05, test_loss 1.717e-01, train_acc 100.0, test_acc 96.8
step 4000, dt 2.90s, train_loss 1.068e-05, test_loss 1.805e-01, train_acc 100.0, test_acc 96.9
step 5000, dt 3.00s, train_loss 5.890e-06, test_loss 1.878e-01, train_acc 100.0, test_acc 96.7
step 6000, dt 2.83s, train_loss 3.352e-06, test_loss 1.952e-01, train_acc 100.0, test_acc 96.7


### Branching Network with 5 nearest neighbors(K)


In [23]:
from models import BranchingNetwork

branching_model = nn.Sequential(
   BranchingNetwork(in_ch = 1, out_ch1 = 16, out_ch2=16, kernel_size = 5), 
   BranchingNetwork(in_ch = 16, out_ch1 = 8, out_ch2=8, kernel_size = 5),
   BranchingNetwork(in_ch = 8, out_ch1 = 4, out_ch2=4, kernel_size = 5),
   nn.Flatten(), 
   nn.Linear(160, 10)
)


# Get summary
from torchsummary import summary 
# summary(branching_model, (1, 40), device='cpu')

In [24]:
branching_results = train_model(data, branching_model, dataset.model_args)

step 1000, dt 25.76s, train_loss 1.018e-01, test_loss 3.166e-01, train_acc 94.8, test_acc 91.9
step 2000, dt 24.62s, train_loss 6.220e-02, test_loss 3.138e-01, train_acc 95.5, test_acc 92.4
step 3000, dt 24.29s, train_loss 5.159e-02, test_loss 2.283e-01, train_acc 97.7, test_acc 94.4
step 4000, dt 23.87s, train_loss 1.929e-02, test_loss 2.250e-01, train_acc 98.0, test_acc 94.9
step 5000, dt 23.67s, train_loss 7.971e-02, test_loss 2.652e-01, train_acc 98.5, test_acc 94.6
step 6000, dt 21.63s, train_loss 1.011e-02, test_loss 2.590e-01, train_acc 97.0, test_acc 93.6


## Regular CNN + Branching NN with different K

In [25]:
# Regular CNN Models 

reg_model_1 = nn.Sequential(
   nn.Conv1d(in_channels=1, out_channels=32, kernel_size=1),
   nn.ReLU(), 
   nn.Conv1d(in_channels=32, out_channels=32, kernel_size=1),
   nn.ReLU(),
   nn.Conv1d(in_channels=32, out_channels=32, kernel_size=1),
   nn.ReLU(), 
   nn.Flatten(), 
   nn.Linear(1280, 10)
)

reg_model_2 = nn.Sequential(
   nn.Conv1d(in_channels=1, out_channels=32, kernel_size=2),
   nn.ReLU(), 
   nn.Conv1d(in_channels=32, out_channels=32, kernel_size=2),
   nn.ReLU(),
   nn.Conv1d(in_channels=32, out_channels=32, kernel_size=2),
   nn.ReLU(), 
   nn.Flatten(), 
   nn.Linear(1184, 10)
)


reg_model_3 = nn.Sequential(
   nn.Conv1d(in_channels=1, out_channels=32, kernel_size=3),
   nn.ReLU(), 
   nn.Conv1d(in_channels=32, out_channels=32, kernel_size=3),
   nn.ReLU(),
   nn.Conv1d(in_channels=32, out_channels=32, kernel_size=3),
   nn.ReLU(), 
   nn.Flatten(), 
   nn.Linear(1088, 10)
)

In [26]:
# summary(reg_model_1, (1, 40), device='cpu')

In [27]:
# summary(reg_model_2, (1, 40), device='cpu')


In [28]:
# summary(reg_model_3, (1, 40), device='cpu')

In [14]:
# Results for Regular CNN models

print("Regular CNN with kernel size 1")
reg_result_1 = train_model(data, reg_model_1, dataset.model_args)
print()

print("Regular CNN with kernel size 2")
reg_result_2 = train_model(data, reg_model_2, dataset.model_args)
print()

print("Regular CNN with kernel size 3")
reg_result_3 = train_model(data, reg_model_3, dataset.model_args)
print()

Regular CNN with kernel size 1
step 1000, dt 2.94s, train_loss 9.094e-01, test_loss 1.356e+00, train_acc 68.5, test_acc 48.7
step 2000, dt 2.77s, train_loss 7.466e-01, test_loss 1.643e+00, train_acc 71.1, test_acc 46.9
step 3000, dt 2.77s, train_loss 6.170e-01, test_loss 2.116e+00, train_acc 71.1, test_acc 47.2
step 4000, dt 2.74s, train_loss 7.430e-01, test_loss 2.120e+00, train_acc 73.8, test_acc 44.5
step 5000, dt 2.74s, train_loss 5.560e-01, test_loss 2.672e+00, train_acc 77.5, test_acc 44.1
step 6000, dt 2.72s, train_loss 5.798e-01, test_loss 2.618e+00, train_acc 74.8, test_acc 44.7

Regular CNN with kernel size 2
step 1000, dt 2.93s, train_loss 2.814e-02, test_loss 4.549e-01, train_acc 99.2, test_acc 91.2
step 2000, dt 2.76s, train_loss 6.512e-02, test_loss 6.773e-01, train_acc 97.8, test_acc 89.5
step 3000, dt 2.78s, train_loss 9.763e-05, test_loss 4.715e-01, train_acc 100.0, test_acc 92.9
step 4000, dt 2.75s, train_loss 4.325e-05, test_loss 4.959e-01, train_acc 100.0, test_acc 

In [29]:
# Models for Branching Network 
branching_model_1 = nn.Sequential(
   BranchingNetwork(in_ch = 1, out_ch1 = 16, out_ch2=16, kernel_size = 1), 
   BranchingNetwork(in_ch = 16, out_ch1 = 8, out_ch2=8, kernel_size = 1),
   BranchingNetwork(in_ch = 8, out_ch1 = 4, out_ch2=4, kernel_size = 1),
   nn.Flatten(), 
   nn.Linear(160, 10)
)

branching_model_2 = nn.Sequential(
   BranchingNetwork(in_ch = 1, out_ch1 = 16, out_ch2=16, kernel_size = 2), 
   BranchingNetwork(in_ch = 16, out_ch1 = 8, out_ch2=8, kernel_size = 2),
   BranchingNetwork(in_ch = 8, out_ch1 = 4, out_ch2=4, kernel_size = 2),
   nn.Flatten(), 
   nn.Linear(160, 10)
)

branching_model_3 = nn.Sequential(
   BranchingNetwork(in_ch = 1, out_ch1 = 16, out_ch2=16, kernel_size = 3), 
   BranchingNetwork(in_ch = 16, out_ch1 = 8, out_ch2=8, kernel_size = 3),
   BranchingNetwork(in_ch = 8, out_ch1 = 4, out_ch2=4, kernel_size = 3),
   nn.Flatten(), 
   nn.Linear(160, 10)
)


In [30]:
# summary(branching_model_1, (1, 40), device='cpu')

In [31]:
# summary(branching_model_2, (1, 40), device='cpu')

In [20]:
# summary(branching_model_3, (1, 40), device='cpu')

In [19]:
# Results for Branching models

print("Branching with K = 1")
branching_results_1 = train_model(data, branching_model_1, dataset.model_args)
print()

print("Branching with K = 2")
branching_results_2 = train_model(data, branching_model_2, dataset.model_args)
print()

print("Branching with K = 3")
branching_results_3 = train_model(data, branching_model_3, dataset.model_args)
print()

Branching with K = 1
step 1000, dt 23.63s, train_loss 1.129e+00, test_loss 1.230e+00, train_acc 60.8, test_acc 49.7
step 2000, dt 21.16s, train_loss 1.096e+00, test_loss 1.301e+00, train_acc 61.8, test_acc 48.4
step 3000, dt 21.45s, train_loss 1.045e+00, test_loss 1.250e+00, train_acc 64.4, test_acc 51.5
step 4000, dt 21.55s, train_loss 9.999e-01, test_loss 1.246e+00, train_acc 64.0, test_acc 49.6
step 5000, dt 20.97s, train_loss 9.725e-01, test_loss 1.263e+00, train_acc 64.2, test_acc 49.2
step 6000, dt 22.52s, train_loss 9.514e-01, test_loss 1.264e+00, train_acc 64.2, test_acc 49.6

Branching with K = 2
step 1000, dt 24.55s, train_loss 3.397e-01, test_loss 6.626e-01, train_acc 85.5, test_acc 79.5
step 2000, dt 24.04s, train_loss 1.860e-01, test_loss 5.470e-01, train_acc 94.2, test_acc 86.9
step 3000, dt 24.11s, train_loss 2.425e-01, test_loss 6.499e-01, train_acc 94.3, test_acc 86.0
step 4000, dt 24.16s, train_loss 1.487e-01, test_loss 6.117e-01, train_acc 95.0, test_acc 88.2
step 50

## Pixel shuffle NN

In [ ]:
pixelshuffle_model = nn.Sequential(
   
   
   
)